## Try it!

Before you start ensure the place the SICK test csv file in the Data/SICK folder.
The file can be found in this github page under this same folder

### Task 1: Classification

In [40]:
import torch
import torch.nn as nn
from torchtext import data
import torch.optim as optim

import pandas as pd
import numpy as np
import sys
from scipy.stats import pearsonr
from collections import Counter

#Reproducing same results
seed = 2020

#Set the seed to be fixed
torch.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

#Global
TEST_BATCH_SIZE = 13

#Load the test set
classif_test_label = data.LabelField(dtype=torch.int64, batch_first=True)
classif_test_sentAB = data.Field(tokenize='spacy', include_lengths=True, batch_first=True)

classif_test_fields = [(None, None), (None, None), (None, None), (None, None), (None, None),
                       ('test_label', classif_test_label), ('test_sentAB', classif_test_sentAB)]

classif_test_dataset = data.TabularDataset(path='../DATA/SICK/SICK test.csv', format='CSV', 
                                           fields=classif_test_fields, skip_header=True)

classif_test_sentAB.build_vocab(classif_test_dataset, min_freq=1, vectors_cache="Vectors/",
                                vectors="glove.6B.300d")

classif_test_label.build_vocab(classif_test_dataset)

classif_test_iterator = data.BucketIterator(classif_test_dataset, TEST_BATCH_SIZE,
                                             sort_key=lambda x : x.classif_test_sentAB,
                                             device=device,
                                             shuffle=False)

In [41]:
class RNNClassifModel(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim,
                 output_dim, num_layers, bidirectional,dropout_rate):
        super().__init__()
        
        #Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #More regularization
        self.dropout1 = nn.Dropout(p=dropout_rate)
        
        #LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,
                            num_layers=num_layers,
                            bidirectional=bidirectional,
                            batch_first=True)
        
        #More regularization
        self.dropout2 = nn.Dropout(p=dropout_rate)
        
        #Full connected layer
        self.fc = nn.Linear(2 * hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        #text = [batch size, max sentence length in batch]
        embedded = self.embedding(text)
        
        #Regularize!
        reg_embedded = self.dropout1(embedded)
        
        #pack the batch sentences to max length
        packed_embedded = nn.utils.rnn.pack_padded_sequence(reg_embedded, text_lengths, 
                                                            batch_first=True, enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        
        inputs = self.dropout2(hidden)
        
        #Direct values are given to CE loss for loss calculation
        #LogSoftmax is used for inference
        outputs = self.fc(inputs)
    
        return outputs

In [42]:
vocab_size = 2298 #Size of the training set vocabulary
embedding_dim = 300
num_hidden_nodes = 32
num_output_nodes = 3
num_layers = 1
bidirectional = True
dropout_rate = 0.4

#Create the model
model = RNNClassifModel(vocab_size, embedding_dim, num_hidden_nodes,
                 num_output_nodes, num_layers, bidirectional,dropout_rate)

#Load the pretrained model
checkpoint = torch.load('../Models/bilstm_task1_fn.pth')
model.load_state_dict(checkpoint["model"])


#Optimizer and Loss
classif_criterion = nn.CrossEntropyLoss()

#Softmax layer only used to get probabilties
softmax = nn.LogSoftmax(dim=1)

model.to(device)

model

RNNClassifModel(
  (embedding): Embedding(2298, 300)
  (dropout1): Dropout(p=0.4, inplace=False)
  (lstm): LSTM(300, 32, batch_first=True, bidirectional=True)
  (dropout2): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=64, out_features=3, bias=True)
)

In [46]:
def classification_prediction(dataset, iterator, criterion):
    iterations = len(dataset) / TEST_BATCH_SIZE

    model.eval()

    running_loss = 0
    running_correct = 0
    _predictions = []
    
    with torch.no_grad():
        for batch_idx, items in enumerate(iterator):
  
            sys.stdout.write('\r')
            sys.stdout.write("{} Iteration :{}/{}"
                                .format("Prediction", batch_idx + 1, iterations))


            #Get the text and length of sentences
            text, text_lengths = items.test_sentAB
            text = text.to(device)
            text_lengths = text_lengths.to(device)

            #Get labels of each batch
            labels = items.test_label
            labels = labels.to(device)

            #Predictions are in size [1, ..]
            outputs = model(text, text_lengths).squeeze()

            #Calculate loss
            loss = criterion(outputs, labels)
            
            #Convert predictions to probabilities
            probabilites = softmax(outputs)
            #Give's the index of the node with the highest probability
            predictions = torch.argmax(probabilites, dim=1)
            correct = (predictions == labels).float()

            #Save the predictions in a list
            _predictions += list(predictions.cpu().squeeze().numpy())
            
            #Calculate running loss and accuracy
            running_loss += loss.item()
            running_correct += correct.sum().item()
        
        epoch_loss = running_loss / len(iterator)
        epoch_accuracy = running_correct / len(dataset)
        
    return epoch_loss, epoch_accuracy, _predictions 

In [47]:
classif_loss, classif_accuracy, classif_predictions = classification_prediction(classif_test_dataset, 
                                                                                classif_test_iterator, 
                                                                                classif_criterion)

Prediction Iteration :379/379.0

In [48]:
#Classification results
classif_loss, classif_accuracy

(0.9488168171653647, 0.565049725999594)

### Task 2: Regression

In [65]:
#Load the test set
regress_test_score = data.Field(dtype=torch.float, use_vocab=False, batch_first=True, 
                         is_target=True, sequential=False)
regress_test_sentAB = data.Field(tokenize='spacy', include_lengths=True, batch_first=True)

regress_test_fields = [(None, None), (None, None), (None, None), ('test_score', regress_test_score),
                        (None, None), (None, None), ('test_sentAB', regress_test_sentAB)]

regress_test_dataset = data.TabularDataset(path='../DATA/SICK/SICK test.csv', format='CSV', 
                                           fields=regress_test_fields, skip_header=True)

regress_test_sentAB.build_vocab(regress_test_dataset, min_freq=1, vectors_cache="Vectors/",
                                vectors="glove.6B.300d")

regress_test_label.build_vocab(regress_test_dataset)

regress_test_iterator = data.BucketIterator(regress_test_dataset, TEST_BATCH_SIZE,
                                             sort_key=lambda x : x.test_sentAB,
                                             device=device,
                                             shuffle=False)

In [66]:
class RNNRegressModel(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim,
                 output_dim, num_layers, bidirectional,dropout_rate):
        super().__init__()
        
        #Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #More regularization
        self.dropout1 = nn.Dropout(p=dropout_rate)
        
        #LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,
                            num_layers=num_layers,
                            bidirectional=bidirectional,
                            batch_first=True)
        
        #More regularization
        self.dropout2 = nn.Dropout(p=dropout_rate)
        
        #Full connected layer
        self.fc = nn.Linear(2 * hidden_dim, output_dim)
        
        #Activation function
        self.act_fn = nn.ReLU()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size, max sentence length in batch]
        embedded = self.embedding(text)
        
        #Regularize!
        reg_embedded = self.dropout1(embedded)
        
        #pack the batch sentences to max length
        packed_embedded = nn.utils.rnn.pack_padded_sequence(reg_embedded, text_lengths, 
                                                            batch_first=True, enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        
        inputs = self.dropout2(hidden)
        
        #Direct values are given to CE loss for loss calculation
        outputs = self.fc(inputs)
        
        outputs = self.act_fn(outputs)
    
        return outputs

In [76]:
vocab_size = 2298 #Size of the training set vocabulary
embedding_dim = 300
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 1
bidirectional = True
dropout_rate = 0.4

#Create the model
regress_model = RNNRegressModel(vocab_size, embedding_dim, num_hidden_nodes,
                                num_output_nodes, num_layers, bidirectional,dropout_rate)

#Load the pretrained model
checkpoint = torch.load('../Models/bilstm_task2_2dropout_1e23_1.pth')
regress_model.load_state_dict(checkpoint["model"])


#Optimizer and Loss
regress_criterion = nn.MSELoss()


model.to(device)

model

RNNRegressModel(
  (embedding): Embedding(2298, 300)
  (dropout1): Dropout(p=0.4, inplace=False)
  (lstm): LSTM(300, 32, batch_first=True, bidirectional=True)
  (dropout2): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=64, out_features=3, bias=True)
  (act_fn): ReLU()
)

In [79]:
def regression_prediction(model, dataset, iterator, criterion):
    iterations = len(dataset) / TEST_BATCH_SIZE

    model.eval()

    running_loss = 0
    running_correct = 0
    predictions = []
    targets = []
    
    
    with torch.no_grad():
        for batch_idx, items in enumerate(iterator):
            
            sys.stdout.write('\r')
            sys.stdout.write("{} Iteration :{}/{}"
                                .format("Prediction", batch_idx + 1, iterations))


            #Get the text and length of sentences
            text, text_lengths = items.test_sentAB
            text = text.to(device)
            text_lengths = text_lengths.to(device)

            #Get labels of each batch
            scores = items.test_score
            scores = scores.to(device)

            #Predictions are in size [1, ..]
            outputs = model(text, text_lengths).squeeze()
            
            #Calculate loss
            loss = criterion(outputs, scores)
            

            #Get the prediction outputs and targets into a list for eval function
            predictions += list(outputs.detach().cpu().numpy())
            targets += list(scores.cpu().numpy())

            
            #Calculate running loss and accuracy
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(iterator)
        
    return epoch_loss, np.array(predictions), np.array(targets)

In [82]:
regress_loss, regress_score, regress_predictions = regression_prediction(regress_model,
                                                                            regress_test_dataset, 
                                                                            regress_test_iterator, 
                                                                            regress_criterion)

Prediction Iteration :379/379.0

In [83]:
regress_loss, regress_score

(1.026610275686259,
 array([3.3450677, 3.4405046, 3.413593 , ..., 3.3067756, 3.8597636,
        3.6444664], dtype=float32))